In [1]:
import sys
sys.path.append("home/daniel/Documents/hsi-forest-segmentation/Kyselica/")
sys.path.append("home/daniel/Documents/hsi-forest-segmentation/Kyselica/src")
sys.path.append('../')
from models.backbones.resnet_encoder import SentinelResNetEncoder

import torch
import torchvision
import numpy as np

import segmentation_models_pytorch as smp

In [2]:
output_path = '/home/daniel/Documents/hsi-forest-segmentation/Kyselica/resources/backbones'

# Save state dict for ResNet from MoCo checkpoint

In [6]:
name = 'resnet18'
resnet_ckp_path = '/home/daniel/Documents/hsi-forest-segmentation/Kyselica/resources/backbones/B13_rn18_moco_0099_ckpt.pth'

In [7]:
net = torchvision.models.resnet18(pretrained=False)
net.conv1 = torch.nn.Conv2d(13, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [8]:
print("=> loading checkpoint '{}'".format(resnet_ckp_path))
checkpoint = torch.load(resnet_ckp_path, map_location="cpu")

# rename moco pre-trained keys
state_dict = checkpoint['state_dict']
#print(state_dict.keys())
for k in list(state_dict.keys()):
    # retain only encoder up to before the embedding layer
    if k.startswith('module.encoder_q') and not k.startswith('module.encoder_q.fc'):
        #pdb.set_trace()
        # remove prefix
        state_dict[k[len("module.encoder_q."):]] = state_dict[k]
    # delete renamed or unused k
    del state_dict[k]

'''
# remove prefix
state_dict = {k.replace("module.", ""): v for k,v in state_dict.items()}
'''
#args.start_epoch = 0
msg = net.load_state_dict(state_dict, strict=False)
#pdb.set_trace()
assert set(msg.missing_keys) == {"fc.weight", "fc.bias"}

print("=> loaded pre-trained model '{}'".format(resnet_ckp_path))

=> loading checkpoint '/home/daniel/Documents/hsi-forest-segmentation/Kyselica/resources/backbones/B13_rn18_moco_0099_ckpt.pth'
=> loaded pre-trained model '/home/daniel/Documents/hsi-forest-segmentation/Kyselica/resources/backbones/B13_rn18_moco_0099_ckpt.pth'


In [9]:
torch.save(state_dict, f"{output_path}/{name}.pth")

In [10]:
print("=> saved pre-trained model '{}'".format(f"{output_path}/{name}.pth"))

=> saved pre-trained model '/home/daniel/Documents/hsi-forest-segmentation/Kyselica/resources/backbones/resnet18.pth'


# Save state dict for ViT from MoCo checkpoint

In [3]:
from models.backbones.vit import VisionTransformerMoCo
import models.backbones.vit as vits
from functools import partial

In [4]:
name = "ViT-S16"
path = '/home/daniel/Documents/hsi-forest-segmentation/Kyselica/resources/backbones/B13_vits16_moco_0099_ckpt.pth'

In [5]:
params = {
    "patch_size": 16,
    "embed_dim": 384,
    "depth": 12,
    "num_heads": 6,
    "mlp_ratio": 4,
    "qkv_bias": True,
    "norm_layer": partial(torch.nn.LayerNorm, eps=1e-6),
    "in_chans": 13,
    "num_classes": 19,
}  
model = VisionTransformerMoCo(**params)

/home/daniel/Documents/hsi-forest-segmentation/.venv/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
x = torch.load(path, map_location="cpu")

In [7]:
linear_keyword = 'head'

print("=> loading checkpoint '{}'".format(path))
checkpoint = torch.load(path, map_location="cpu")

# rename moco pre-trained keys
state_dict = checkpoint['state_dict']
for k in list(state_dict.keys()):
    # retain only base_encoder up to before the embedding layer
    if k.startswith('module.base_encoder') and not k.startswith('module.base_encoder.%s' % linear_keyword):
        # remove prefix
        state_dict[k[len("module.base_encoder."):]] = state_dict[k]
    # delete renamed or unused k
    del state_dict[k]

msg = model.load_state_dict(state_dict, strict=False)
print(msg)
assert set(msg.missing_keys) == {"%s.weight" % linear_keyword, "%s.bias" % linear_keyword}

print("=> loaded pre-trained model '{}'".format(name))

=> loading checkpoint '/home/daniel/Documents/hsi-forest-segmentation/Kyselica/resources/backbones/B13_vits16_moco_0099_ckpt.pth'
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
=> loaded pre-trained model 'ViT-S16'


In [9]:
model

VisionTransformerMoCo(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(13, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Iden

In [8]:
torch.save(state_dict, f"{output_path}/{name}.pth")
print("=> saved pre-trained model '{}'".format(f"{output_path}/{name}.pth"))

=> saved pre-trained model '/home/daniel/Documents/hsi-forest-segmentation/Kyselica/resources/backbones/ViT-S16.pth'


# Registe and test the encoder

In [8]:
from functools import partial


smp.encoders.encoders['ViT-S/16_S2'] = {
    'encoder': VisionTransformerMoCo,
    'pretrained_settings': {
        'SSL4EO-S12': {
            'url': 'file:///home/daniel/Documents/hsi-forest-segmentation/Kyselica/resources/backbones/ViT-S16.pth',
        }
    },
    'params': {
        "patch_size": 16,
        "embed_dim": 384,
        "depth": 12,
        "num_heads": 6,
        "mlp_ratio": 4,
        "qkv_bias": True,
        "norm_layer": partial(torch.nn.LayerNorm, eps=1e-6),
        "in_chans": 13,
        "num_classes": 19,
    }    
}


In [9]:
model = smp.DeepLabV3(encoder_name="ViT-S/16_S2", encoder_weights='SSL4EO-S12', in_channels=13, classes=11)

In [10]:
test_input = torch.randn(10, 13, 224, 224)

In [11]:
model(test_input).shape

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [19]

In [1]:
import json

In [2]:
path = "/media/daniel/data/sentinel2/Belgium_autum2022/grid_features.json"

In [3]:
with open(path, 'r') as f:
    grid_features = json.load(f)

In [12]:
len(grid_features['features'])

3132